In [ ]:
def loadImages(path):
    # Put files into lists and return them as one list of size 4
    image_files = sorted([os.path.join(path, file)
         for file in os.listdir(path)])
    return image_files

# Display one image
def display_one(a, title1="Original"):
    plt.imshow(a), plt.title(title1)
    plt.xticks([]), plt.yticks([])
    plt.show()


# Display two images
def display(a, b, title1="Original", title2="Edited"):
    plt.subplot(121), plt.imshow(a), plt.title(title1)
    plt.xticks([]), plt.yticks([])
    plt.subplot(122), plt.imshow(b), plt.title(title2)
    plt.xticks([]), plt.yticks([])
    plt.show()


# Preprocessing
def processing(data):
    # loading image
    # Getting 3 images to work with
    img = [cv2.imread('./posters/' + i, cv2.IMREAD_UNCHANGED) for i in data[:len(data)]]
    # --------------------------------
    # setting dim of the resize
    height = 220
    width = 220
    dim = (width, height)
    res_img = []
    original = []
    no_noise = []
    results = {}
    for i in range(len(img)):
        #print(i)
        res = cv2.resize(img[i], dim, interpolation=cv2.INTER_LINEAR)
        res_img.append(res)

    # Visualizing one of the images in the array
        original.append(res_img[i])
        if i == 0:
            display_one(original[i])

    # ----------------------------------
    # Remove noise
    # Gaussian
    #for i in range(len(res_img)):
        blur = cv2.GaussianBlur(res_img[i], (5, 5), 0)
        no_noise.append(blur)


        image = no_noise[i]
        if i == 0:
            display(original[i], image, 'Original', 'Blurred')
    #---------------------------------

        # Segmentation
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

        # Displaying segmented images
        if i == 0:
            display(original[i], thresh, 'Original', 'Segmented')
        results[i] = thresh
    return(results)
def main():
    # calling global variable
    global image_path

    # import the data set that include the attributes for each picture with 0-1 representation, setting 'image_id' as index
    df_attr = pd.read_csv('./MovieGenre.csv', encoding = 'latin-1')
    df_attr.set_index('imdbId', inplace=True)
    df_attr.replace(to_replace=-1, value=0, inplace=True)  # replace -1 by 0
    
    
    df = pd.read_csv('./MovieGenre.csv', encoding = 'latin-1')
    saved_column = df.Genre
    drama = []
    partition = []
    for x in saved_column:
        if 'Horror' in str(x):
            drama.append(1)
        else:
            drama.append(0)
        partition = -1
    df_attr['Horror'] = drama
    df_attr['Partition'] = partition
    df_par_attr = df_attr

    random.seed(101)  ##so we hopefully get some data in our partitions

    # number of observations we want in each group
    NUM_OF_TRAIN = 1000
    NUM_OF_TEST = 500

    # just is just making the default -1 for everything, so if an observation doesnt get picked, it wont get used later
    df_par_attr.loc[:, 'partition'] = -1

    # 0 -> TRAINING
    # 1 -> TEST

    # setting training data
    train_pos_ids = df_par_attr[(df_par_attr['Horror'] == 1) & (df_par_attr['partition'] == -1)].sample(
        NUM_OF_TRAIN // 2).index
    df_par_attr.loc[train_pos_ids, 'partition'] = 0
    train_neg_ids = df_par_attr[(df_par_attr['Horror'] == 0) & (df_par_attr['partition'] == -1)].sample(
        NUM_OF_TRAIN // 2).index
    df_par_attr.loc[train_neg_ids, 'partition'] = 0

    # setting test data
    test_pos_ids = df_par_attr[(df_par_attr['Horror'] == 1) & (df_par_attr['partition'] == -1)].sample(
        NUM_OF_TEST // 2).index
    df_par_attr.loc[test_pos_ids, 'partition'] = 1
    test_neg_ids = df_par_attr[(df_par_attr['Horror'] == 0) & (df_par_attr['partition'] == -1)].sample(
        NUM_OF_TEST // 2).index
    df_par_attr.loc[test_neg_ids, 'partition'] = 1

    # display counter by partition
    df_par_attr['partition'].value_counts().sort_index()
    train_ids = df_par_attr['partition'].values == 0
    test_ids = df_par_attr['partition'].values == 1

    '''The var Dataset is a list with all images in the folder '''
    dataset = loadImages(image_path)

    idxs = np.where(train_ids)[0]
    trnx = [dataset[i] for i in idxs]
    #print("List of files in the folder:\n", trnx[:len(trnx)])
    #print("--------------------------------")

    # sending all the images to pre-processing
    pro_trnx = processing(trnx)

    y = df_attr["Horror"]
    trny = [y[i] for i in idxs]
    pro_trnx = np.array(list(pro_trnx.values()))
    nsamples, nx, ny = pro_trnx.shape
    pro_trnx = pro_trnx.reshape((nsamples, nx * ny))

    logis = LogisticRegression().fit(pro_trnx,trny)

    idxs = np.where(test_ids)[0]
    tstx = [dataset[i] for i in idxs]
    tsty = [y[i] for i in idxs]
    pro_tstx = processing(tstx)
    pro_tstx = np.array(list(pro_tstx.values()))
    nsamples, nx, ny = pro_tstx.shape
    pro_tstx = pro_tstx.reshape((nsamples, nx * ny))

    predy = logis.predict(pro_tstx)
    

    print("logistic")
    tn, fp, fn, tp = confusion_matrix(tsty,predy).ravel()
    print("True Negative Rate", tn/250)
    print("False Positive Rate", fp/250)
    print("False Negative Rate", fn/250)
    print("True Positive Rate", tp/250)
    print("Specificity", 1-fp/250)
    print("Sensitivity", tp/250)
    print("Accuracy", sum(predy == tsty) / len(tsty))

    
    clf2 = svm.LinearSVC()
    print(clf2.fit(pro_trnx,trny))
    predysvm2 = clf2.predict(pro_tstx)
    tn, fp, fn, tp = confusion_matrix(tsty,predysvm2).ravel()
    print("True Negative Rate", tn/250)
    print("False Positive Rate", fp/250)
    print("False Negative Rate", fn/250)
    print("True Positive Rate", tp/250)
    print("Specificity", 1-fp/250)
    print("Sensitivity", tp/250)
    print("Accuracy", sum(predysvm2 == tsty) / len(tsty))
    fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
    print("AUROC", metrics.auc(fpr, tpr))
    
    D = [i for i in np.arange(2, 5, 1)]
    
    for d in D:
        clf3 = svm.LinearSVC()
        print(clf3.fit(pro_trnx,trny))
        predysvm3 = clf3.predict(pro_tstx)
        tn, fp, fn, tp = confusion_matrix(tsty,predysvm3).ravel()
        print("True Negative Rate", tn/250)
        print("False Positive Rate", fp/250)
        print("False Negative Rate", fn/250)
        print("True Positive Rate", tp/250)
        print("Specificity", 1-fp/250)
        print("Sensitivity", tp/250)
        print("Accuracy", sum(predysvm3 == tsty) / len(tsty))
        fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
        print("AUROC", metrics.auc(fpr, tpr))
    print("done")
main()